In [1]:
!pip install pandas

In [1]:
import numpy as np
import scipy as sp
from scipy import linalg
import math
from scipy.linalg import qr
import pandas as pd

In [2]:
from TSVDMII_v2 import tSVDMII
from TSVDMII_v2 import mode3 
from TSVDMII_v2 import mode3i 
from TSVDMII_v2 import fronorm

Quick Test

In [3]:
# Example tensor
tensor = np.random.rand(120, 160, 120)
n3 = tensor.shape[0]

# Perform QR decomposition on the random matrix
random_matrix = np.random.rand(n3, n3)
q, r = qr(random_matrix)

class Transform:
    def __init__(self, n3):
        self.L = q @ np.diag(np.sign(np.diag(r)))
        self.l = n3
        self.inverseL = self.L.T

# Create an instance of the Transform class
transform_instance = Transform(n3)

U = transform_instance.L

[m,p,n] = tensor.shape
[r,q] = U.shape 

M = U
A = tensor 
nrg = 0.95

S = tSVDMII(A, M, 0.98, 1)

Relative error in approximation is 0.14140418869388405


Importing x_mmgks_1 from demo dynamic Emoji and reshaping into a 128 by 128 by 33 tensor

In [4]:
x_mmgks = np.load('x_mmgks.npy')
tensor = x_mmgks.reshape((33, 128, 128), order='F')

# Convert the NumPy array to a pandas DataFrame
df = pd.DataFrame(x_mmgks)

# Save the DataFrame to a CSV file
#df.to_csv('x_mmgks.csv', index=False)

Performing compression on the tensor 

In [7]:
[m,p,n] = tensor.shape
T3 = np.reshape(tensor, (m,p*n), order='F')

# Perform SVD
U, _, _ = np.linalg.svd(T3, full_matrices=False)
M = U.T

# Compress 
nrg = 0.99
tensorcmp = tSVDMII(tensor, M, nrg, 1)
print(tensorcmp[1])
print(tensorcmp[2])

Relative error in approximation is 0.09991087997181869
0.09991087997181869
5.12621359223301


In [8]:
# Code testing

# Create a 2x2x5 tensor with orthogonal frontal slices
A = np.zeros((5, 2, 2), dtype=int)

# Fill the tensor with orthogonal frontal slices
A[0, :, :] = [[1, 0], [0, 1]]
A[1, :, :] = [[0, 1], [1, 0]]
A[2, :, :] = [[1, 0], [0, 1]]
A[3, :, :] = [[0, 1], [1, 0]]
A[4, :, :] = [[1, 0], [0, 1]]

[m,p,n] = A.shape

# reshaping A 
A3 = np.reshape(A, (m,p*n), order='F')

# Perform SVD
U, _, _ = np.linalg.svd(A3, full_matrices=False)
M = U.T
[r,q] = M.shape 

# Performing tSVDM Compression
nrg = 0.98 
Ahat = mode3(A, M) 
[m,p,n] = Ahat.shape

Uah = np.zeros((m, p, min(p, n)))
Vah = np.zeros((m, n, min(p, n)))
Sah = np.zeros((m, min(p, n), min(p, n)))
Delta = np.zeros((min(p, n), m)) 

for i in np.arange(0, m):
    U, S, Vt = np.linalg.svd(Ahat[i, :, :], full_matrices=False)

    Uah[i, :, :] = U
    Sah[i, :, :] = np.diag(S)
    Vah[i, :, :] = Vt.T
    Delta[:, i] = S**2

frnm2 = np.sum(Delta.ravel()) # jth col of Delta are the squared svals of jth frontal slice
sortedDeltas = np.sort(Delta.ravel())[::-1] # sort these globally, descending order 
cus = np.cumsum(sortedDeltas)
cus = cus/frnm2 # cumulative sum of the sorted deltas. These are called the energies 
fi = np.where(cus<=nrg)[0] # find all indices where energy is less than the prescribed energy value
idx = fi.shape[0] # Zeros the values of energy that are larger than prescribed value
cutoff = sortedDeltas[idx]
fi = np.where(sortedDeltas >= cutoff)[0] # find all values in the delta array that are greater than cutoff

Sah2 = np.zeros_like(Sah)
F = np.where(Sah>=np.sqrt(cutoff))
Sah2[F] = Sah[F]
Sah = Sah2 # this is a facewise diagonal with too small svalues commented out 
del_count = np.count_nonzero(Sah) # Count the total number of non-zeros 

ln = m*p*n 
ln2 = p*del_count + n*del_count
cmplvl = ln/ln2

Appx = np.zeros((m, p, n))
for i in np.arange(0,m):
    Appx[i,:,:] = Uah[i,:,:]@Sah[i,:,:]@Vah[i,:,:].T

# Applying mode3i product 
Appx = mode3i(Appx, M, 1)
RE = fronorm(Appx - A) / fronorm(A)
print("Relative error in approximation is", RE)



Relative error in approximation is 1.3136335981433191e-16
